# Node
Node is the basic building block of the WorkTree. A node has inputs, outputs, and the executor. A node executor can be a `calcfunction`, `workfunction`, `calcjob`, `Workchain` or any other Python function. A node can be created in three ways.

## Decorator

Decorate any Python function using the `node` decorator. To use the power of AiiDA (e.g. save the results to a database, keep provenance), one can use the `node.calcfunction` decorator.


In [44]:
from aiida_worktree import node
from aiida import orm

# define add node
@node()
def add(x, y):
   return x + y

# define multiply calcfunction node
@node.calcfunction()
def multiply(x, y):
   return orm.Float(x + y)

The input sockets are generated automatically based on the function arguments. The default name of the output socket is `result`. There are also some built-in sockets, like `_wait` and `_outputs`.  One can create a node instance and inpsect its inputs and outputs:

In [45]:
add1 = add.node()
print("Inputs:", add1.inputs.keys())
print("Outputs:", add1.outputs.keys())

Inputs: ['x', 'y', '_wait']
Outputs: ['result', '_wait', '_outputs']


If you want to change the name of the output sockets, or if there are more than one output. You can define the outputs explicitly.
For example: `["General", "sum"]`, where the first value `General` indicates the data type, and the second value is the name of the socket. The data type tell the code how to display the socket in the GUI and to serialize data into database. We use `General` for any data type.

In [46]:
# define add calcfunction node
@node(outputs=[["General", "sum"],
               ["General", "difference"]])
def add_minus(x, y):
   return {"sum": x + y, "difference": x - y}

print("Inputs:", add_minus.node().inputs.keys())
print("Outputs:", add_minus.node().outputs.keys())

Inputs: ['x', 'y', '_wait']
Outputs: ['sum', 'difference', '_wait', '_outputs']


Then, one can use the node inside the WorkTree:

In [47]:
from aiida_worktree import WorkTree
wt = WorkTree()
add_minus1 = wt.nodes.new(add_minus, name="add_minus1")
multiply1 = wt.nodes.new(multiply, name="multiply1")
wt.links.new(add_minus1.outputs["sum"], multiply1.inputs["x"])

NodeLink(from="add1.result", to="multiply1.x")

## Build from AiiDA
One can build a node from an already existing AiiDA component: `calcfunction`,  `workfunction`, `calcjob`, `Workchain` with the `build_node` function.

In [48]:
from aiida_worktree import build_node
ndata = {"path": "aiida.calculations.arithmetic.add.ArithmeticAddCalculation"}
AddNode = build_node(ndata)

The inputs and outputs of the node are automatically generated based on the AiiDA process.

In [49]:
node = AddNode()
print("Inputs:")
for input in node.inputs:
   if "." not in input.name:
      print(f"  - {input.name}")
print("Outputs:")
for output in node.outputs:
   if "." not in output.name:
      print(f"  - {output.name}")

Inputs:
  - metadata
  - code
  - monitors
  - remote_folder
  - x
  - y
  - _wait
Outputs:
  - remote_folder
  - remote_stash
  - retrieved
  - sum
  - _outputs
  - _wait


## Define a Node
Create a node class by inheriting from `Node` base class.


In [55]:
from aiida_worktree.node import Node

class MyAdd(Node):

    identifier: str = "MyAdd"
    name = "MyAdd"
    node_type = "calcfunction"
    catalog = "Test"
    kwargs = ["x", "y"]

    def create_sockets(self):
        self.inputs.clear()
        self.outputs.clear()
        inp = self.inputs.new("General", "x")
        inp.add_property("General", "x", default=0.0)
        inp = self.inputs.new("General", "y")
        inp.add_property("General", "y", default=0.0)
        self.outputs.new("General", "sum")

    def get_executor(self):
        return {
            "path": "aiida_worktree.test",
            "name": "add",
        }

Then, one can use the node by using its identifier.

In [56]:
from aiida_worktree import WorkTree
wt = WorkTree()
wt.nodes.new(MyAdd, name="add1")


MyAdd(name="add1", properties = [], inputs = ["x", "y"], outputs = ["sum"])

One can also register the node in node pool, and then use its `identifer` directly.
```python
wt.nodes.new("MyAdd", name="add1")
```